In [2]:
!pip install -q spacy pandas unidecode
!python -m spacy download es_core_news_lg


     ---------------------------------------- 0.0/568.0 MB ? eta -:--:--
     ---------------------------------------- 0.0/568.0 MB ? eta -:--:--
     -------------------------------------- 0.1/568.0 MB 550.5 kB/s eta 0:17:12
     -------------------------------------- 0.1/568.0 MB 939.4 kB/s eta 0:10:05
     -------------------------------------- 0.2/568.0 MB 919.0 kB/s eta 0:10:18
     -------------------------------------- 0.2/568.0 MB 798.5 kB/s eta 0:11:52
     ---------------------------------------- 0.3/568.0 MB 1.1 MB/s eta 0:08:54
     ---------------------------------------- 0.4/568.0 MB 1.3 MB/s eta 0:07:12
     ---------------------------------------- 0.5/568.0 MB 1.3 MB/s eta 0:07:13
     ---------------------------------------- 0.6/568.0 MB 1.4 MB/s eta 0:06:41
     ---------------------------------------- 0.6/568.0 MB 1.3 MB/s eta 0:07:06
     ---------------------------------------- 0.7/568.0 MB 1.4 MB/s eta 0:06:46
     ---------------------------------------- 0.8/568.

In [3]:
import pandas as pd
import numpy as np
import re

from unidecode import unidecode
import spacy

# Load Spanish large model
nlp = spacy.load("es_core_news_lg")


In [4]:
train = pd.read_csv("C:/Users/Rodri/OneDrive/Rodri estudioso/GitHub/taller_3/stores/train.csv")
test  = pd.read_csv("C:/Users/Rodri/OneDrive/Rodri estudioso/GitHub/taller_3/stores/test.csv")

train.head()


,property_id,city,price,month,year,surface_total,surface_covered,rooms,bedrooms,bathrooms,property_type,operation_type,lat,lon,title,description
0,4fa04a187235d7509a43a8cf,Bogotá D.C,580000000.0,10,2020,NaN,NaN,3.0,3,NaN,Casa,Venta,4.707211,-74.099645,de casa en los cerezos,cod 2901fr461 de casa multifamiliar 4 pisos ga...
1,f9cfbb306af543a759c1f3f2,Bogotá D.C,600000000.0,10,2020,NaN,NaN,9.0,9,NaN,Casa,Venta,4.612200,-74.131642,casa en floralia bogota,casa en o en el barrio floralia en la ciudad d...
2,957a84e8e93ae2b33bc6f61b,Bogotá D.C,400000000.0,10,2020,NaN,NaN,8.0,8,NaN,Casa,Venta,4.586620,-74.138664,de casa en el barrio fatima,de casa de 7 mt2 de frente por 20 mt2 de fondo...
3,eb32d75b32bc5e8677207fc7,Bogotá D.C,800000000.0,10,2020,NaN,NaN,6.0,6,NaN,Casa,Venta,4.678147,-74.142154,casa fontibon a media cuadra de avda ferrocarril,oportunidad salio a la una amplia casa rentabl...
4,880a6e7bd74dc2568940d275,Bogotá D.C,480000000.0,10,2020,NaN,NaN,NaN,3,3.0,Casa,Venta,4.756503,-74.051553,casa en bogota villa del prado,casa villa del prado amplia estilo asturiana m...


In [5]:
## Limpiamos el texto para el patrón regex

def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.lower()
    text = unidecode(text)  # remove accents
    # replace non-alphanumeric by space
    text = re.sub(r"[^0-9a-zA-Z]+", " ", text)
    # collapse whitespace
    text = re.sub(r"\s+", " ", text).strip()
    return text

for df in (train, test):
    df["description_raw"] = df["description"].fillna("")
    df["description_clean"] = df["description_raw"].apply(clean_text)


In [7]:
## Corremos spacy en el raw

def spacy_tokenize(df, text_col="description_raw", id_col="property_id"):
    """
    Run spaCy on a dataframe column and return a tidy token table.
    """
    rows = []
    texts = df[text_col].fillna("").tolist()
    ids   = df[id_col].tolist()

    for prop_id, doc in zip(ids, nlp.pipe(texts, batch_size=64, n_process=1)):
        sent_id = 0
        for sent in doc.sents:
            for token in sent:
                rows.append({
                    "property_id": prop_id,
                    "sent_id": sent_id,
                    "token": token.text,
                    "lemma": token.lemma_,
                    "pos": token.pos_,
                    "ent_type": token.ent_type_
                })
            sent_id += 1

    return pd.DataFrame(rows)

train_tokens = spacy_tokenize(train, text_col="description_raw", id_col="property_id")
train_tokens.head()

test_tokens = spacy_tokenize(test, text_col="description_raw", id_col="property_id")


In [8]:
train_tokens["lemma"].value_counts().head(50)



lemma
de               149892
y                 99015
con               98852
el                90817
en                73818
bano              39954
uno               38583
zona              38543
apartamento       34172
a                 33762
piso              31463
sala              29738
comedor           26892
cocina            26160
dos               24233
para              23700
2                 22064
amplio            19824
br                19802
contar            19236
parqueadero       19196
habitación        19190
social            17888
excelente         17771
3                 17263
alcoba            16909
cerca             14892
estudio           14298
principal         14028
independiente     13929
mucho             13711
deposito          13576
tres              13482
bao               13296
ubicado           12760
garaje            12737
casa              12632
él                12309
acceso            11844
integral          11809
terraza           11615
edificio  

In [9]:
mask_noun = train_tokens["pos"] == "NOUN"
train_tokens.loc[mask_noun, "lemma"].value_counts().head(50)


lemma
zona           36298
apartamento    33698
piso           27433
sala           24796
bano           20529
cocina         19776
habitación     19190
parqueadero    18401
alcoba         15692
acceso         11796
garaje         11479
casa           10970
terraza        10758
edificio       10707
parque         10298
deposito        9811
nivel           9023
estudio         8638
centro          8604
chimenea        8528
espacio         7395
via             6966
calle           6906
ropa            6805
sector          6686
vista           6650
conjunto        6632
bbq             6514
gas             6433
madera          6250
ascensor        5798
vigilancia      5597
visitante       5489
servicio        5483
area            5058
comedor         4903
gimnasio        4541
cuarto          4462
transporte      4258
lavanderia      4230
patio           4188
barrio          3994
carrera         3903
hora            3766
cancha          3619
bao             3617
seguridad       3557
acabado

In [10]:
train_tokens.loc[train_tokens["pos"] == "ADJ", "lemma"].value_counts().head(50)


lemma
amplio           19806
excelente        17758
comedor          15873
principal        14025
independiente    13717
ubicado          12752
integral         11617
comercial        11165
social           10231
privado          10187
exterior          7826
abierto           7675
cubierto          7671
buen              7308
comunal           6511
cerrado           6494
natural           6419
iluminado         5902
hermoso           5847
infantil          5534
segundo           5308
cuarto            5275
remodelado        4808
primero           4804
laminado          4709
residencial       4658
verde             4341
cocín             4050
construido        3950
lindo             3776
moderno           3680
espectacular      3408
gimnasio          3158
americano         3053
gran              2797
interior          2777
publico           2657
cercano           2654
exclusivo         2587
tercer            2514
mayor             2466
tranquilo         2459
apto              2379
auxil

In [ ]:
import re
import numpy as np
import pandas as pd

# ==========
# DICCIONARIO DE LEMMAS
# ==========

PROPERTY_TYPE = {
    "apartment": ["apartamento", "apartaestudio", "duplex", "loft", "penthouse"],
    "house": ["casa"],
}
# Quitar el código
def remove_codes(text: str) -> str:
    """
    Elimina patrones tipo 'codigo 3459', 'código 3459', 'cod 2332', 'ref 12345'
    del texto. Devuelve un string limpio.
    """
    if pd.isna(text):
        return ""
    t = str(text).lower()

    # quitar 'codigo 1234', 'código 1234', 'cod 1234', 'ref 1234'
    t = re.sub(r"\b(cod(igo)?|codigo|código|ref|referencia)\s*\d{2,6}\b", " ", t)
    t = re.sub(r"\s+", " ", t).strip()
    return t

train["description_clean"] = train["description_clean"].apply(remove_codes)
test["description_clean"]  = test["description_clean"].apply(remove_codes)

ROOM_WORDS      = ["habitacion", "alcoba", "habitación", "dormitorio", "habitaciones", "alcobas"]
BATH_WORDS      = ["bano", "baos", "banos" ]
GARAGE_WORDS    = ["garaje", "parqueadero"]
GARAGE_ADJ      = ["cubierto", "independiente", "cerrado", "visitantes"]
STORAGE_WORDS   = ["deposito", "bodega"]
TERRACE_WORDS   = ["terraza"]
BALCONY_WORDS   = ["balcon"]
KITCHEN_WORDS   = ["cocina"]
KITCHEN_ADJ     = ["integral", "abierto"]
LAUNDRY_WORDS   = ["ropa", "lavandera", "lavanderia"]
SECURITY_WORDS  = ["seguridad", "vigilancia", "vigilante", "porteria", "portería", "portero"]

# Proximidad centrada en centros comerciales (no uso comercial)
CENTRO_COMERCIAL_PATTERNS = [
    r"\bcentro comercial\b",
    r"\bcentros comerciales\b",
]

# Uso comercial (del inmueble)
USO_COMERCIAL_PATTERNS = [
    r"\buso comercial\b",
    r"\blocal comercial(es)?\b",
    r"\barea comercial\b",
    r"\bzonas? comerciales?\b",
    r"\bsector comercial\b",
    r"\binmueble comercial\b",
]

regex_cc  = re.compile("|".join(CENTRO_COMERCIAL_PATTERNS))
regex_uso = re.compile("|".join(USO_COMERCIAL_PATTERNS), flags=re.IGNORECASE)

# ==========
# NÚMEROS EN ESPAÑOL BÁSICOS
# ==========

SPANISH_NUM_WORDS = {
    "uno": 1, "una": 1, "un": 1,
    "dos": 2,
    "tres": 3,
    "cuatro": 4,
    "cinco": 5,
    "seis": 6,
    "siete": 7,
    "ocho": 8,
    "nueve": 9,
    "diez": 10,
}

#Para el piso donde está el apartamento:

FLOOR_ORDINAL_WORDS = {
    "primer": 1,
    "primero": 1,
    "segundo": 2,
    "tercer": 3,
    "tercero": 3,
    "cuarto": 4,
    "quinto": 5,
    "sexto": 6,
    "septimo": 7,
    "séptimo": 7,
    "octavo": 8,
    "noveno": 9,
    "decimo": 10,
    "décimo": 10,
}


def token_to_int(tok: str):
    """Convierte un token como '2' o 'dos' en int. Si no puede, devuelve None."""
    if tok is None:
        return None
    tok = tok.lower()
    tok = tok.strip()
    if tok.isdigit():
        return int(tok)
    return SPANISH_NUM_WORDS.get(tok)
# Patrón largo casa
AREA_PATTERN = re.compile(
    r"\b(\d{2,4})\s*(m2|m 2|mts2|mts|mt2|mtrs2|mtrs|metros cuadrados|metros)\b"
)

def extract_count_from_text(text: str,
                            keywords_regex: str,
                            min_val: int,
                            max_val: int) -> float:
    """
    Extrae un conteo (baños, habitaciones, etc.) de un texto:
    - keywords_regex: patrón de palabras (ej: 'bano|baños|banos')
    - aplica filtro de plausibilidad [min_val, max_val]
    - si hay varios candidatos, prioriza los plausibles;
      si solo hay absurdos -> NaN.
    """
    if pd.isna(text):
        return np.nan
    t = str(text).lower()

    candidates = []

    # 1) número antes: '3 banos'
    pattern_num_before = rf"\b(\d{{1,2}})\s+(?:{keywords_regex})\b"
    for m in re.finditer(pattern_num_before, t):
        candidates.append(int(m.group(1)))

    # 2) número después: 'banos 3'
    pattern_num_after = rf"\b(?:{keywords_regex})\s+(\d{{1,2}})\b"
    for m in re.finditer(pattern_num_after, t):
        candidates.append(int(m.group(1)))

    # 3) número en palabras antes: 'tres banos'
    words_pattern = "|".join(SPANISH_NUM_WORDS.keys())
    pattern_word_before = rf"\b({words_pattern})\s+(?:{keywords_regex})\b"
    for m in re.finditer(pattern_word_before, t):
        word = m.group(1)
        val = SPANISH_NUM_WORDS.get(word)
        if val is not None:
            candidates.append(val)

    if not candidates:
        return np.nan

    # --- Filtro de plausibilidad ---
    plausible = [c for c in candidates if min_val <= c <= max_val]

    # 1) Si hay plausibles -> usamos uno (el máximo, por ejemplo)
    if plausible:
        return max(plausible)

    # 2) Si solo hay absurdos -> NaN
    return np.nan

In [49]:
def add_token_index(tokens: pd.DataFrame) -> pd.DataFrame:
    tokens = tokens.sort_values(["property_id", "sent_id"]).copy()
    tokens["tok_id"] = tokens.groupby("property_id").cumcount()
    return tokens

train_tokens = add_token_index(train_tokens)
test_tokens  = add_token_index(test_tokens)


In [51]:
## Funciones de length
def add_length_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df["n_palabras_desc"] = df["description_clean"].fillna("").str.split().str.len()
    df["n_palabras_title"] = df["title"].fillna("").str.split().str.len()
    
    # Versión sin códigos tipo "cod 1234"
    title_nocod = df["title"].fillna("").str.lower()
    title_nocod = title_nocod.str.replace(r"\bcod(igo)?\s*\d+\b", "", regex=True)
    title_nocod = title_nocod.str.replace(r"\s+", " ", regex=True).str.strip()
    df["n_palabras_title_nocod"] = title_nocod.str.split().str.len()
    return df


In [52]:
## Área total

def extract_area_m2(text: str):
    """
    Extrae los metros cuadrados mencionados en el texto.
    Devuelve el máximo valor encontrado (int) o np.nan si no hay match.
    """
    if pd.isna(text):
        return np.nan
    t = text.lower()
    matches = AREA_PATTERN.findall(t)
    if not matches:
        return np.nan

    # matches es lista de tuples (numero_str, unidad)
    nums = [int(m[0]) for m in matches]
    if not nums:
        return np.nan
    return max(nums)


def add_area_feature(df: pd.DataFrame, text_col: str = "description_clean") -> pd.DataFrame:
    """
    Añade una columna 'area_text_m2' con el área en m2 extraída del texto.
    """
    out = df.copy()
    out["area_text_m2"] = out[text_col].apply(extract_area_m2)
    return out

In [53]:
## Tipo propiedad
def add_property_type_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(list)
    
    def classify_type(lemmas):
        lemmas = set(lemmas)
        is_ap = any(l in lemmas for l in PROPERTY_TYPE["apartment"])
        is_house = any(l in lemmas for l in PROPERTY_TYPE["house"])
        if is_ap and not is_house:
            return "apartment"
        elif is_house and not is_ap:
            return "house"
        elif is_ap and is_house:
            return "ambiguous"
        else:
            return "unknown"
    
    tipo = lemmas_by_prop.apply(classify_type)
    df = df.merge(tipo.rename("tipo_inmueble_text"), on="property_id", how="left")
    return df


In [69]:
# Sala comedor habitaciones baños
def add_room_features(df: pd.DataFrame,
                      tokens: pd.DataFrame,
                      text_col: str = "description_clean") -> pd.DataFrame:
    df = df.copy()
    
    # ================================
    # 1) Flags sala / comedor (igual que antes)
    # ================================
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(list)
    
    df["tiene_sala"] = df["property_id"].map(
        lambda pid: int(
            any(
                word in set(lemmas_by_prop.get(pid, []))
                for word in ["sala", "zonasocial", "zona social"]
            )
        )
    )

    df["tiene_comedor"] = df["property_id"].map(
        lambda pid: int("comedor" in set(lemmas_by_prop.get(pid, [])))
    )
    
    # sala comedor conj.
    def has_sala_comedor(text):
        if pd.isna(text):
            return 0
        t = text.lower()
        return int("sala comedor" in t or "salacomedor" in t or "sala-comedor" in t)
    
    df["sala_comedor_conjunto"] = df["description_raw"].apply(has_sala_comedor)
    
    # ================================
    # 2) Conteo aproximado de habitaciones/baños usando tokens (tu lógica original)
    # ================================
    tokens_rooms = tokens[tokens["lemma"].isin(ROOM_WORDS + BATH_WORDS)].copy()
    
    # Juntamos el token previo como posible número
    num_candidates = tokens[["property_id", "tok_id", "token"]].copy()
    num_candidates["num_value"] = num_candidates["token"].map(token_to_int)
    
    merged = tokens_rooms.merge(
        num_candidates,
        left_on=["property_id", "tok_id"],
        right_on=["property_id", "tok_id"],
        suffixes=("", "_self"),
        how="left",
    )
    # número inmediatamente anterior
    prev = num_candidates.copy()
    prev["tok_id"] = prev["tok_id"] + 1  # para hacer join con el anterior
    prev = prev.rename(columns={"num_value": "num_prev"})
    
    merged = merged.merge(
        prev[["property_id", "tok_id", "num_prev"]],
        on=["property_id", "tok_id"],
        how="left",
    )
    
    def pick_num(row):
        if not np.isnan(row.get("num_prev", np.nan)):
            return row["num_prev"]
        return row.get("num_value", np.nan)
    
    merged["num_detected"] = merged.apply(pick_num, axis=1)
    
    # habitaciones: ROOM_WORDS
    mask_room = merged["lemma"].isin(ROOM_WORDS)
    room_counts = merged[mask_room].groupby("property_id")["num_detected"].sum(min_count=1)
    
    # si no hay número detectado, al menos poner 1 si se mencionan habitaciones
    freq_room = tokens[tokens["lemma"].isin(ROOM_WORDS)].groupby("property_id").size()
    
    n_rooms = room_counts.combine(
        freq_room,
        func=lambda n, f: n if not np.isnan(n) else float(f) if not pd.isna(f) else np.nan
    )
    
    # baños
    mask_bath = merged["lemma"].isin(BATH_WORDS)
    bath_counts = merged[mask_bath].groupby("property_id")["num_detected"].sum(min_count=1)
    freq_bath = tokens[tokens["lemma"].isin(BATH_WORDS)].groupby("property_id").size()
    n_baths = bath_counts.combine(
        freq_bath,
        func=lambda n, f: n if not np.isnan(n) else float(f) if not pd.isna(f) else np.nan
    )
    
    df = df.merge(n_rooms.rename("n_habitaciones_text"), on="property_id", how="left")
    df = df.merge(n_baths.rename("n_banos_text"), on="property_id", how="left")
    
    # ================================
    # 3) Filtro de plausibilidad + fallback con extract_count_from_text
    # ================================
    # Nota: text_col debería ser 'description_clean' ya sin códigos.
    room_keywords_regex = "|".join(re.escape(w) for w in ROOM_WORDS)
    bath_keywords_regex = "|".join(re.escape(w) for w in BATH_WORDS)

    # Recuento alternativo desde el texto plano
    rooms_from_text = df[text_col].apply(
        lambda txt: extract_count_from_text(
            txt,
            keywords_regex=room_keywords_regex,
            min_val=1,
            max_val=18
        )
    )

    baths_from_text = df[text_col].apply(
        lambda txt: extract_count_from_text(
            txt,
            keywords_regex=bath_keywords_regex,
            min_val=1,
            max_val=18
        )
    )

    # Si el valor original es plausible, lo dejamos.
    # Si es NaN o está fuera de rango, usamos el valor desde el texto.
    df["n_habitaciones_text"] = np.where(
        df["n_habitaciones_text"].between(1, 18),
        df["n_habitaciones_text"],
        rooms_from_text
    )

    df["n_banos_text"] = np.where(
        df["n_banos_text"].between(1, 18),
        df["n_banos_text"],
        baths_from_text
    )
    
    return df


In [55]:
# Piso del apto


def extract_floor_from_text(text: str):
    """Extrae el piso de la descripción usando varias reglas y filtra valores raros."""
    if text is None or (isinstance(text, float) and np.isnan(text)):
        return np.nan

    t = str(text).lower()
    floors = []

    # 1) 'piso 3', 'piso 10' → número DESPUÉS de 'piso'
    for m in re.finditer(r"\bpiso\s+(\d{1,2})\b", t):
        floors.append(int(m.group(1)))

    # 2) 'tercer piso', 'quinto piso', etc.
    for m in re.finditer(
        r"\b(primer|primero|segundo|tercer|tercero|cuarto|quinto|sexto|septimo|séptimo|octavo|noveno|decimo|décimo)\s+piso\b",
        t
    ):
        word = m.group(1)
        val = FLOOR_ORDINAL_WORDS.get(word)
        if val is not None:
            floors.append(val)

    # 3) SOLO si no encontramos nada antes, mirar '3 piso'
    if not floors:
        for m in re.finditer(r"\b(\d{1,2})\s+piso\b", t):
            floors.append(int(m.group(1)))

    if not floors:
        return np.nan

    # 4) Filtro de plausibilidad para eliminar cosas tipo 51, 75, etc.
    floors = [f for f in floors if 1 <= f <= 40]
    if not floors:
        return np.nan

    # Puedes usar min, max o el primero; yo tomo el mínimo por seguridad
    return min(floors)


def add_floor_feature(df: pd.DataFrame, text_col: str = "description_clean") -> pd.DataFrame:
    """
    Añade una columna 'piso_text' con el número de piso detectado.
    Solo se aplica a filas clasificadas como apartment.
    """
    out = df.copy()
    # inicializamos todo como NaN
    out["piso_text"] = np.nan

    # filtramos solo apartamentos
    mask_ap = out["tipo_inmueble_text"] == "apartment"

    out.loc[mask_ap, "piso_text"] = out.loc[mask_ap, text_col].apply(extract_floor_from_text)

    return out


In [56]:
# Garaje parqueo
def add_garage_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    
    # ¿aparece alguna palabra de garaje?
    has_garage = (
        tokens[tokens["lemma"].isin(GARAGE_WORDS)]
        .groupby("property_id")
        .size()
        .gt(0)
        .astype(int)
    )
    df = df.merge(has_garage.rename("tiene_garaje_text"), on="property_id", how="left")
    df["tiene_garaje_text"] = df["tiene_garaje_text"].fillna(0).astype(int)
    
    # Conteo aproximado de # garajes usando números
    tokens_garage = tokens[tokens["lemma"].isin(GARAGE_WORDS)].copy()
    num_candidates = tokens[["property_id", "tok_id", "token"]].copy()
    num_candidates["num_value"] = num_candidates["token"].map(token_to_int)
    
    merged = tokens_garage.merge(
        num_candidates,
        on=["property_id", "tok_id"],
        how="left",
        suffixes=("", "_self"),
    )
    
    prev = num_candidates.copy()
    prev["tok_id"] = prev["tok_id"] + 1
    prev = prev.rename(columns={"num_value": "num_prev"})
    
    merged = merged.merge(
        prev[["property_id", "tok_id", "num_prev"]],
        on=["property_id", "tok_id"],
        how="left",
    )
    
    def pick_num(row):
        if not np.isnan(row.get("num_prev", np.nan)):
            return row["num_prev"]
        return row.get("num_value", np.nan)
    
    merged["num_detected"] = merged.apply(pick_num, axis=1)
    
    n_garages = merged.groupby("property_id")["num_detected"].sum(min_count=1)
    df = df.merge(n_garages.rename("n_garajes_text"), on="property_id", how="left")
    
    # Flags de cubierto / independiente
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(set)
    
    df["garaje_cubierto_text"] = df["property_id"].map(
        lambda pid: int(
            ("cubierto" in lemmas_by_prop.get(pid, set()))
            and df.loc[df["property_id"] == pid, "tiene_garaje_text"].iloc[0] == 1
        )
    )
    df["garaje_indep_text"] = df["property_id"].map(
        lambda pid: int("independiente" in lemmas_by_prop.get(pid, set()))
    )
    
    return df


In [57]:
def add_storage_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    has_storage = (
        tokens[tokens["lemma"].isin(STORAGE_WORDS)]
        .groupby("property_id")
        .size()
        .gt(0)
        .astype(int)
    )
    df = df.merge(has_storage.rename("tiene_deposito_bodega"), on="property_id", how="left")
    df["tiene_deposito_bodega"] = df["tiene_deposito_bodega"].fillna(0).astype(int)
    return df


In [58]:
# Terraza
def add_terrace_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(set)
    
    df["tiene_terraza"] = df["property_id"].map(
        lambda pid: int(any(l in lemmas_by_prop.get(pid, set()) for l in TERRACE_WORDS))
    )
    df["tiene_balcon"] = df["property_id"].map(
        lambda pid: int(any(l in lemmas_by_prop.get(pid, set()) for l in BALCONY_WORDS))
    )
    
    # terraza propia / privada (muy básico)
    def terraza_propia(pid):
        lem = lemmas_by_prop.get(pid, set())
        return int("terraza" in lem and ("privado" in lem or "propio" in lem))
    
    df["terraza_propia"] = df["property_id"].map(terraza_propia)
    return df


In [59]:
#cocina, área de limpieza
def add_kitchen_laundry_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(set)
    
    df["tiene_cocina"] = df["property_id"].map(
        lambda pid: int("cocina" in lemmas_by_prop.get(pid, set()))
    )
    
    def cocina_integral(pid):
        lem = lemmas_by_prop.get(pid, set())
        return int("cocina" in lem and "integral" in lem)
    
    def cocina_abierta(pid):
        lem = lemmas_by_prop.get(pid, set())
        return int("cocina" in lem and "abierto" in lem)
    
    df["cocina_integral"] = df["property_id"].map(cocina_integral)
    df["cocina_abierta"]  = df["property_id"].map(cocina_abierta)
    
    # Patio / zona de ropas: heurística simple
    def tiene_laundry(text):
        if pd.isna(text):
            return 0
        t = text.lower()
        if any(p in t for p in ["zona de ropas", "patio de ropas", "lavanderia", "lavandera"]):
            return 1
        return 0
    
    df["tiene_patio_ropas"] = df["description_raw"].apply(tiene_laundry)
    return df


In [60]:
#Remodelado

def add_remodel_features(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    def flag_remodel(text):
        if pd.isna(text):
            return 0
        t = text.lower()
        # positivo
        if any(p in t for p in ["recien remodelado", "recien remodelada", "remodelado", "remodelada", "actualizado"]):
            # pero no marcar si solo dice "para remodelar"
            if "para remodelar" in t:
                return 0
            return 1
        # explícitamente "para remodelar" = 0 pero podrías guardar otra flag
        return 0
    
    df["remodelada_text"] = df["description_raw"].apply(flag_remodel)
    return df


In [61]:
# Seguridad 
def add_security_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    lemmas_by_prop = tokens.groupby("property_id")["lemma"].apply(set)
    
    def has_security(pid):
        lem = lemmas_by_prop.get(pid, set())
        if any(w in lem for w in SECURITY_WORDS):
            return 1
        return 0
    
    df["tiene_vigilancia_text"] = df["property_id"].map(has_security)
    return df


In [62]:
#temas comerciales
def add_commercial_features(df: pd.DataFrame, text_col: str = "description_clean") -> pd.DataFrame:
    df = df.copy()
    text = df[text_col].fillna("")
    
    df["menciona_centro_comercial"] = text.apply(
        lambda t: int(bool(regex_cc.search(t)))
    )
    df["uso_comercial_text"] = text.apply(
        lambda t: int(bool(regex_uso.search(t)))
    )
    return df


In [63]:
#Cercanía
def add_proximity_features(df: pd.DataFrame, text_col: str = "description_clean") -> pd.DataFrame:
    df = df.copy()
    txt = df[text_col].fillna("").str.lower()
    
    df["menciona_cercania_txt"] = txt.apply(
        lambda t: int(any(p in t for p in ["cerca a", "cerca de", "frente a", "a pocos minutos"]))
    )
    
    df["cercania_bus_text"] = txt.apply(
        lambda t: int(any(p in t for p in ["transmilenio", "estacion", "estación", "paradero", "autopista norte", "transporte publico", "transporte"]))
    )
    
    df["cercania_cc"] = df["menciona_centro_comercial"]  # de la función anterior
    
    df["cercania_cafe_txt"] = txt.apply(
        lambda t: int(any(p in t for p in ["cafe", "café", "restaurante", "bares"]))
    )
    
    # 'a X cuadras'
    df["menciona_cuadras_txt"] = txt.apply(
        lambda t: int(bool(re.search(r"a\s+\d+\s+cuadras", t)))
    )
    
    return df


In [64]:
def add_text_features(df: pd.DataFrame, tokens: pd.DataFrame) -> pd.DataFrame:
    """
    df: DataFrame con columnas:
        - property_id
        - title
        - description_raw
        - description_clean
    tokens: DataFrame con columnas:
        - property_id
        - sent_id
        - tok_id
        - token
        - lemma
        - pos
    """
    out = df.copy()
    out = add_length_features(out)
    out = add_property_type_features(out, tokens)
    out = add_room_features(out, tokens)
    out = add_garage_features(out, tokens)
    out = add_storage_features(out, tokens)
    out = add_terrace_features(out, tokens)
    out = add_kitchen_laundry_features(out, tokens)
    out = add_remodel_features(out)
    out = add_security_features(out, tokens)
    out = add_commercial_features(out, text_col="description_clean")
    out = add_proximity_features(out, text_col="description_clean")
    out = add_floor_feature(out, text_col="description_clean")
    out = add_area_feature(out,  text_col="description_clean")
    
    return out


In [70]:
train_enriched = add_text_features(train, train_tokens)
test_enriched  = add_text_features(test,  test_tokens)



In [73]:

def combine_counts_set2(df: pd.DataFrame,
                        col_raw: str,
                        col_text: str,
                        min_val: int,
                        max_val: int,
                        new_col: str) -> pd.DataFrame:
    """
    Combina una columna 'raw' del dataset (ej. bedrooms)
    con una columna derivada del texto (ej. n_habitaciones_text)
    usando la lógica del Set 2.
    """
    out = df.copy()

    raw = out[col_raw].astype("float")
    txt = out[col_text].astype("float")

    final = pd.Series(np.nan, index=out.index, dtype="float")

    # 1) Ambos presentes
    both_mask = raw.notna() & txt.notna()
    diff = (raw - txt).abs()

    # 1a) Si están cerca (<= 1), tomamos el máximo (más informativo)
    close_mask = both_mask & (diff <= 1)
    final.loc[close_mask] = np.maximum(raw[close_mask], txt[close_mask])

    # 1b) Si están muy lejos (> 1) y raw es plausible, usamos raw
    far_mask = both_mask & (diff > 1)
    raw_valid_far = far_mask & raw.between(min_val, max_val)
    final.loc[raw_valid_far] = raw[raw_valid_far]

    # 1c) Si están muy lejos y raw NO es plausible pero txt sí → usamos txt
    txt_valid_far = far_mask & ~raw.between(min_val, max_val) & txt.between(min_val, max_val)
    final.loc[txt_valid_far] = txt[txt_valid_far]

    # 2) Sólo raw presente y plausible
    raw_only = raw.notna() & txt.isna() & raw.between(min_val, max_val) & final.isna()
    final.loc[raw_only] = raw[raw_only]

    # 3) Sólo txt presente y plausible
    txt_only = txt.notna() & raw.isna() & txt.between(min_val, max_val) & final.isna()
    final.loc[txt_only] = txt[txt_only]

    # (cualquier otro caso queda como NaN)
    out[new_col] = final
    return out


In [75]:
def combine_bedrooms_set2(df: pd.DataFrame) -> pd.DataFrame:
    # Bedrooms plausibles: 1–14
    return combine_counts_set2(
        df,
        col_raw="bedrooms",
        col_text="n_habitaciones_text",
        min_val=1,
        max_val=14,
        new_col="bedrooms_final_set2"
    )

def combine_bathrooms_set2(df: pd.DataFrame) -> pd.DataFrame:
    # Baños plausibles: 1–14
    return combine_counts_set2(
        df,
        col_raw="bathrooms",
        col_text="n_banos_text",
        min_val=1,
        max_val=14,
        new_col="bathrooms_final_set2"
    )

In [80]:
train_enriched = combine_bedrooms_set2(train_enriched)
train_enriched = combine_bathrooms_set2(train_enriched)

test_enriched  = combine_bedrooms_set2(test_enriched)
test_enriched  = combine_bathrooms_set2(test_enriched)

In [92]:
train_enriched["tipo_inmueble_text_std"] = (
    train_enriched["tipo_inmueble_text"]
    .astype(str)
    .str.lower()
    .replace({
        "apartment": "Apartamento",
        "apartamento": "Apartamento",
        "house": "Casa",
        "casa": "Casa"
    })
)

test_enriched["tipo_inmueble_text_std"] = (
    test_enriched["tipo_inmueble_text"]
    .astype(str)
    .str.lower()
    .replace({
        "apartment": "Apartamento",
        "apartamento": "Apartamento",
        "house": "Casa",
        "casa": "Casa"
    })
)

# Convertir "none", "unknown", "ambiguous", "nan", etc. → NaN
train_enriched["tipo_inmueble_text_std"] = train_enriched["tipo_inmueble_text_std"].replace(
    {"none": np.nan, "nan": np.nan, "unknown": np.nan, "ambiguous": np.nan}
)

test_enriched["tipo_inmueble_text_std"] = test_enriched["tipo_inmueble_text_std"].replace(
    {"none": np.nan, "nan": np.nan, "unknown": np.nan, "ambiguous": np.nan}
)


In [93]:
def combine_property_type_set2(df: pd.DataFrame,
                               col_raw: str = "property_type",
                               col_text: str = "tipo_inmueble_text_std",
                               new_col: str = "property_type_final") -> pd.DataFrame:

    raw = df[col_raw]
    txt  = df[col_text]

    final = pd.Series("unknown", index=df.index, dtype="object")

    # ambos y coinciden
    mask_agree = raw.notna() & txt.notna() & (raw == txt)
    final[mask_agree] = raw[mask_agree]

    # solo texto
    mask_only_text = raw.isna() & txt.notna()
    final[mask_only_text] = txt[mask_only_text]

    # solo raw
    mask_only_raw = raw.notna() & txt.isna()
    final[mask_only_raw] = raw[mask_only_raw]

    # conflicto → raw
    mask_conflict = raw.notna() & txt.notna() & (raw != txt)
    final[mask_conflict] = raw[mask_conflict]

    df[new_col] = final
    return df


In [ ]:
train_enriched = combine_property_type_set2(train_enriched)
test_enriched  = combine_property_type_set2(test_enriched)


In [96]:
#Guardamos la vaina:

train_enriched.to_csv("train_enriched2.csv", index=False, encoding="utf-8")
test_enriched.to_csv("test_enriched2.csv", index=False, encoding="utf-8")